In [ ]:
"""
Dedalus script solving for the eigenvalues of the Mathieu equation. This script
demonstrates solving a periodic eigenvalue problem with nonconstant coefficients
and produces a plot of the Mathieu eigenvalues 'a' as a function of the
parameter 'q'. It should take just a few seconds to run (serial only).

We use a Fourier basis to solve the EVP:
    dx(dx(y)) + (a - 2*q*cos(2*x))*y = 0
where 'a' is the eigenvalue. Periodicity is enforced by using the Fourier basis.

To run and plot:
    $ python3 mathieu_evp.py
"""

import numpy as np
import matplotlib.pyplot as plt
import dedalus.public as d3
import logging
logger = logging.getLogger(__name__)


# Parameters
N = (1e-5)**(0.5) # stratification
theta = 5*10**(-3)
f = 10**(-4)
S = theta*N/f
gamma = (1+S**2)**(-1)
Gshear = (theta*N**2*gamma)/f
n = f/Gshear
Ri = N**2/(Gshear**2)
alpha = (N**2*theta)/(f*Gshear)
k = (2*np.pi)/2000
delta_list =  np.linspace(0, 1, 10)
t = gamma**(0.5)*np.pi/2
uz = np.sin(gamma**(-0.5)*t)
vz = np.cos(gamma**(-0.5)*t)-1
bz = np.cos(gamma**(-0.5)*t)-1

In [ ]:
N**2*0.5*(200**2-103**2)

In [ ]:
# Basis
coord = d3.Coordinate('z')
dist = d3.Distributor(coord, dtype=np.complex128)
basis = d3.Legendre(coord, 56, bounds=(0, 103),dealias=3/2)

In [ ]:
# Fields
u = dist.Field(bases=basis)
v = dist.Field(bases=basis)
w = dist.Field(bases=basis)
b = dist.Field(bases=basis)
p = dist.Field(bases=basis)
tau_1 = dist.Field(name="tau_1")
tau_2 = dist.Field(name="tau_2")
tau_3 = dist.Field(name="tau_3")
omega = dist.Field()

In [ ]:
# Substitutions
z = dist.local_grid(basis)
delta = dist.Field()
one_z = dist.Field(bases=basis)
one_z['g'] = 1-z
lift_basis = basis.derivative_basis(1)
lift = lambda A: d3.Lift(A,lift_basis,-1)
dz = lambda A: d3.Differentiate(A, coord)
wz = dz(w)+lift(tau_1)+lift(tau_3)
pz = dz(p)+lift(tau_2)

In [ ]:
# Problem
problem = d3.EVP([u,v,w,b,p,tau_1,tau_2,tau_3], eigenvalue=omega, namespace=locals())
problem.add_equation("-1j*omega*u-gamma**(0.5)*delta*uz*w+1j*k*gamma**(0.5)*delta*uz*one_z*u+1j*k*Ri*p-alpha*b = 0")
problem.add_equation("-1j*omega*v-(1+delta+delta*gamma*vz)*w+1j*k*gamma**(0.5)*delta*uz*one_z*v+u-n*theta*w= 0")
problem.add_equation("-1j*omega*n**2*w+1j*k*gamma**(0.5)*delta*n*uz*one_z*w+n*theta*v+Ri*pz-Ri*b = 0")
problem.add_equation("1j*k*u+wz=0")
problem.add_equation("-1j*omega*b+u*(alpha)/(Ri)+(1-gamma-gamma*delta*bz)*w+1j*k*gamma**(0.5)*delta*uz*one_z*b = 0")
problem.add_equation("w(z=0)=0")
problem.add_equation("w(z=103)+u*theta=0")
problem.add_equation("p(z=103)=0")

In [ ]:
# Solver
solver = problem.build_solver()
evals = []
for deltai in delta_list:
    delta['g'] = deltai
    solver.solve_dense(solver.subproblems[0], rebuild_matrices=True)
    sorted_evals = np.sort(solver.eigenvalues.imag)
    evals.append(sorted_evals[:10])
evals = np.array(evals)


In [ ]:
evals*1e-4

In [ ]:
solver.subproblems[0]

In [ ]:
# Plot
fig = plt.figure(figsize=(6, 4))
plt.plot(delta_list, evals[:, 0::2], '.-', c='C0')
plt.plot(delta_list, evals[:, 1::2], '.-', c='C1')
plt.xlim(delta_list.min(), delta_list.max())
# plt.ylim(-10, 30)
# plt.xlim(0.4,0.6)
plt.xlabel("delta")
plt.ylabel("eigenvalues")
plt.title("Mathieu eigenvalues")
plt.tight_layout()
# plt.savefig("mathieu_eigenvalues.png", dpi=200)